In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
import tqdm as nootbook_tqdm
from sklearn.ensemble import AdaBoostRegressor


c:\Users\sunrise\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df=pd.read_csv("sunny_cluster.csv")
df.head()

,Clearsky DHI,GHI,cluster,Temperature,Clearsky DNI,Relative Humidity,Wind Speed,DNI,Cloud Type,Clearsky GHI,DHI,Solar Zenith Angle
0,108,403,1,14.5,889,40.65,3.4,241,7,618,265,54.95
1,114,702,1,15.7,920,36.14,3.7,920,0,702,114,50.20
2,115,715,1,16.4,923,33.37,3.9,923,0,715,115,49.41
3,111,657,1,16.7,901,31.61,4.0,901,1,657,111,52.76
4,100,533,1,16.6,854,30.95,4.0,854,0,533,100,59.55


In [3]:
df.shape

(2872, 12)

In [4]:
df.isnull().sum()

Clearsky DHI          0
GHI                   0
cluster               0
Temperature           0
Clearsky DNI          0
Relative Humidity     0
Wind Speed            0
DNI                   0
Cloud Type            0
Clearsky GHI          0
DHI                   0
Solar Zenith Angle    0
dtype: int64

In [5]:
df.drop(['cluster'], axis = 1, inplace = True) 

In [6]:
df.describe()

,Clearsky DHI,GHI,Temperature,Clearsky DNI,Relative Humidity,Wind Speed,DNI,Cloud Type,Clearsky GHI,DHI,Solar Zenith Angle
count,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000,2872.000000
mean,196.069290,732.918872,28.169394,744.552228,23.098088,4.163022,722.517409,0.430362,744.193593,201.089833,40.970286
std,70.114563,193.575924,8.048195,134.986357,11.223323,1.540044,164.395314,1.591959,189.422882,73.647434,15.695106
min,79.000000,320.000000,7.500000,162.000000,4.460000,0.300000,17.000000,0.000000,358.000000,79.000000,5.210000
25%,148.000000,579.000000,21.200000,666.000000,15.297500,3.100000,644.000000,0.000000,600.000000,149.000000,30.105000
50%,181.000000,734.000000,29.500000,770.000000,20.280000,4.200000,760.000000,0.000000,746.000000,184.000000,44.480000
75%,225.000000,899.250000,35.200000,844.000000,29.010000,5.200000,839.000000,0.000000,908.000000,233.000000,53.142500
max,520.000000,1086.000000,44.000000,1008.000000,79.390000,8.700000,1008.000000,8.000000,1086.000000,520.000000,66.840000


In [7]:
s1= MinMaxScaler(feature_range=(0,1))
X= s1.fit_transform(df)

In [8]:
s2=MinMaxScaler(feature_range=(0,1))
y= s2.fit_transform(df[['GHI']])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42,shuffle=False)

In [10]:
model = AdaBoostRegressor(n_estimators=100,learning_rate=0.1)
model.fit(X_train, y_train)
print(model)

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


AdaBoostRegressor(learning_rate=0.1, n_estimators=100)


In [11]:
import time
start=time.time()
prediction = model.predict(X_test)
end=time.time()
continental= end-start
print("inference time:",continental)

inference time: 0.02796316146850586


In [12]:
prediction.shape

(575,)

In [13]:
prediction = prediction.reshape(prediction.shape[0], 1)

In [14]:
prediction.shape

(575, 1)

In [15]:
Pred= s2.inverse_transform(prediction)

In [16]:
Actual= s2.inverse_transform(y_test)

In [17]:
Calculated = pd.DataFrame(Pred, columns = ['Prediction'])

In [18]:
Calculated.to_csv(r'E:\CatBoost_1\Johannesburg\AdaBoost\sunny hours\prediction.csv', index = False)

In [19]:
Actual = pd.DataFrame(Actual, columns = ['Actual'])

In [20]:
Actual.to_csv(r'E:\CatBoost_1\Johannesburg\AdaBoost\sunny hours\Actual.csv', index = False)

In [21]:
x=Pred
z=Actual

In [23]:
from sklearn.metrics import mean_squared_error
import math

 
MSE = mean_squared_error(z,x)
 
RMSE = math.sqrt(MSE)
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(z,x)
MAE
print("Root Mean Square Error:",RMSE)
print("Mean Square Error:", MSE)
print("Mean Absolute Error:",MAE)


Root Mean Square Error: 18.470981318656857
Mean Square Error: 341.17715087417065
Mean Absolute Error: 16.252604383953674


In [24]:
max= x.max()
max

946.3861111111115

In [25]:
min=x.min()
min

402.98947368421057

In [26]:
NRMSE= (RMSE/(max-min))*100
print("Normalized Root Mean Square Error:",NRMSE)

Normalized Root Mean Square Error: 3.3991710743962082
